# Homework 3
##### Bayesian Machine Learning; DS 6040
##### Jacqueline Unciano

In [1]:
# Analytics and modeling
import pymc as pm
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_predict
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from scipy import special, stats
import statsmodels.api as sm


# Graphics and visualization
import matplotlib.pyplot as plt
import seaborn as sns
import arviz as az
az.style.use('arviz-darkgrid')
color = '#87ceeb'

# Random seed
random_seed = 730612

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


ModuleNotFoundError: No module named 'sklearn'

### READ ME

Hi Navya~~~ I gave up :)

The code doesn't work and even when I copied the code from my peers to see if my code was just wrong... it still didn't work. I don't get it. How is it going to run ok once, and then the next time I run it, it throws me an error? Where was that error before? I'm at my wits end and so I gave up. Even my explanations don't really make any sense, I'm so... frustrated QAQ. I'm only turning this in because maybe I can get like 10% on it or something lmao.

### Honor Code

I have neither given nor received any unauthorised aid on this assignment.

--Jacqui Unciano

## Question 1

(10) Based on the analysis we did in class comparing systolic blood pressure among men in the South African study [1], you are now asked to
extend this analysis by comparing low density lipoprotein (ldl) levels between the same two groups in the study: those with and without coronary
heart disease. Perform this analysis and comment on whether you find a
statistically significant difference between the groups for this variable. To
do this analysis you will use the CHD data set (CHDdata.cv).


In [ ]:
data1 = pd.read_csv("C:/Users/jacqu/Downloads/algae.csv")

In [ ]:
data1.head()

In [ ]:
data1.describe()

In [ ]:
df1 = data1[["ldl", "chd"]]
df1.hist("ldl", by="chd", figsize=(12, 4))

In [ ]:
y0 = df1.loc[df1["chd"]==0]['ldl']
y1 = df1.loc[df1["chd"]==1]["ldl"]

In [ ]:
random_seed = 134536
cores = 4
μ_prior = df1['ldl'].mean()
σ_prior = df1['ldl'].std() * 2

# prior for Std
σ_low = 1
σ_high = 100

with pm.Model() as model:
    CHD0_mean = pm.Normal("CHD0_mean", mu=μ_prior, sigma=σ_prior)
    CHD1_mean = pm.Normal("CHD1_mean", mu=μ_prior, sigma=σ_prior)
    CHD0_std = pm.Uniform("CHD0_std", lower=σ_low, upper=σ_high)
    CHD1_std = pm.Uniform("CHD1_std", lower=σ_low, upper=σ_high)
    ν = pm.Exponential("ν", 1/29)
    CHD0 = pm.StudentT("No_CHD", nu=ν, mu=CHD0_mean, sigma=CHD0_std, observed=y0)
    CHD1 = pm.StudentT("CHD", nu=ν, mu=CHD1_mean, sigma=CHD1_std, observed=y1)
    diff_of_means = pm.Deterministic("difference of means", CHD0_mean - CHD1_mean)
    diff_of_stds = pm.Deterministic("difference of stds", CHD0_std - CHD1_std)
    effect_size = pm.Deterministic(
        "effect size", diff_of_means / np.sqrt((CHD0_std ** 2 + CHD1_std ** 2) / 2)
    )

    chd_trace=pm.sample(random_seed = random_seed, cores = cores)

In [ ]:
az.plot_trace(chd_trace, kind = "rank_bars")

In [ ]:
az.summary(chd_trace)

In [ ]:
az.plot_forest(chd_trace, var_names = ['CHD0_mean','CHD1_mean'])

### Answer 1

Yes, there is a statistically significant difference in ldl between those with chd and those without chd.

## Question 2

(30) High concentrations of certain types of algae can cause algae blooms
in rivers and lakes. These blooms can have deleterious effects on the
wildlife in the environment in the area to include infecting the drinking
water used by human populations. The European Union has a goal to
develop methods that can monitor water for harmful algae concentrations
and predict algae blooms. They would like to do this using just chemical
tests since these are quicker and cheaper than biological analysis which is
more accurate. The data come from the COIL international data analysis
competition (1999) and consist of 200 samples of water from the same
river over a period of 4 months. There are 11 predictor variables that
describe the season of the year when the samples were collected, the size
and speed of the river at time of collection, and 8 chemical m.

For each of these predictor variable observations, the data set also contains
measurements of 7 different harmful algae, labeled a1-a7 but not named.
For this work we will use the a1 measurements as the response variable.
With algae.csv data and the stated goal of the EU perform the following
tasks and answer the stated questions.asurements:

In [ ]:
algae_df = pd.read_csv("C:\Users\jacqu\Downloads\algae.csv")

In [ ]:
algae_df.head()

(a) Perform exploratory data analysis (EDA) to include as minimum a
review of the summary statistics for the data. Briefly explain how
this EDA informs the next steps of your work.

In [ ]:
col_names = algae_df.columns
col_names

In [ ]:
algae_df = algae_df.loc[:,['season', 'size', 'speed', 'mxPH', 'mnO2', 'Cl', 'NO3', 'NH4', 'oPO4',
       'PO4', 'Chla', 'a1']]

In [ ]:
algae_df.describe()

In [ ]:
# Pair plots of the chemical predictors and the algae, a1

#plt.hist(algae_clean_df['mxPH'])
sns.pairplot(algae_df,vars= ['mxPH' ,'mnO2' ,'Cl' ,'NO3' ,'NH4' ,'oPO4' ,'PO4' ,'Chla', 'a1' ])

In [ ]:
# Several nonlinear relationships - e.g., Chla
# log transform

fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (10,8))
ax1.scatter(algae_df['Chla'], algae_df['a1'])
ax2.scatter(np.log(algae_df['Chla']+.1), np.log(algae_df['a1']+1))
ax1.set_title("Raw Data")
ax2.set_title("Log transformed")

In [ ]:
# Exploratory analysis of categorical variables

sns.boxplot(data=algae_df, x="season", y="a1", hue = 'size')

In [ ]:
df = algae_df[["speed", "a1"]]
print(df.groupby("speed").agg(['mean', 'std', 'count']))
sns.kdeplot(x="a1", hue="speed", data=df)

In [ ]:
# Focus just on a1 as response

az.plot_dist(algae_df['a1'], kind = 'hist')

In [ ]:
az.plot_dist(np.log10(algae_df['a1']+.8), kind = 'hist')

### Answer 2.A

The graphs of the categorical variables show this distribution of the data for each category. Generally, the data follows the same distribution(I'm thinking normal or T distribution). Knowing the data's distribution will also help with choosing the prior parameters when building the model. The plots for the numericald variable helps with thinking about what kind of transformations we can do with the data (most likely log transformations since that would be easier to interpret concerning our model).

(b) Clean the data by identifying missing values and use the following
strategy to estimate missing values: 1. For continuous (real-valued)
data use the mean; 2. For discrete (integer valued) use the median;
and 3. For categorical or nominal data use the mode. Then scale the
predictor variables (subtract the mean and divide by the standard
deviation) and code the categorical variables. Explain why you are
doing this. Reparameterize the response variable using a log transform and explain why you are doing this.

In [ ]:
algae_df.isna().sum()

In [ ]:
# We need to code the categorical variables
# Use one-hot encoding
# Getting dummy variables for categorical variables

log_a1 = np.log10(algae_df.a1+1)

algae_df.drop(['a1'], axis = 1, inplace = True)

algae_model_df = pd.get_dummies(algae_df, drop_first = False, dtype = float)
algae_model_df.describe(include = 'all')

In [ ]:
algae_df.isna().sum()

In [ ]:
# This class imputes the missing values as
# (1) most frequent if the variable is categorical
# (2) mean if the variable is real (floating point)
# (3) median if the variable is an integer

# Here is a class that will provide imputation
# This is an extension by D.Brown to sveitser, 2014 https://stackoverflow.com/users/469992/sveitser

from sklearn.base import TransformerMixin

class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value
        in column.

        Columns of dtype floating point are imputed with the mean.

        Columns of other types are imputed with median of the column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O')
                               else X[c].mean() if X[c].dtype == np.dtype('f')
                                else X[c].median() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

In [ ]:
# Impute missing values

algae_clean_df = DataFrameImputer().fit_transform(algae_model_df)
algae_clean_df.isnull().sum()

### Answer 2.B

We are scaling the predictor variables because interpreting the model can be difficult if the predictors are all different scaling. So we normalise the predictors so that it's easier to interpret the results.

(c) Build a robust Bayesian regression model to predict the response and
explain your choices for the priors.

In [ ]:
algae_clean_df.corr()

In [ ]:
# Create log transformed predictors for a nonlinear model
nl_set = ['Cl','PO4','Chla']
nl_names = ['Cl_nl','PO4_nl','Chla_nl']

for n,m in zip(nl_set,nl_names):
    algae_clean_df[m] = np.log(algae_clean_df[n]+1)

In [ ]:
#Notice the values of the predictors imply the need for scaling
# Use Z-scaling

numeric_var = ['mxPH', 'mnO2', 'Cl', 'NO3', 'NH4', 'PO4', 'Chla']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(algae_clean_df[numeric_var])
stats.describe(X_scaled)

In [ ]:
#Nonlinear variable scaling
# Use Z-scaling

nl_numeric_var = ['mxPH', 'mnO2', 'Cl_nl', 'NO3', 'NH4', 'PO4_nl', 'Chla_nl']
scaler = StandardScaler()
XNL_scaled = scaler.fit_transform(algae_clean_df[nl_numeric_var])
stats.describe(XNL_scaled)

In [ ]:
algae_clean_df.columns

In [ ]:
categorical_var = algae_clean_df.iloc[:,7:17].columns

algae_clean_df[categorical_var].to_numpy()
X_algae = np.hstack((X_scaled, algae_clean_df[categorical_var].to_numpy()))

# Data for linear regression

categorical_var = algae_clean_df.iloc[:,7:17].columns
X_algae = np.hstack((X_scaled, algae_clean_df[categorical_var].to_numpy()))

# Data for nonlinear regression

XNL_algae = np.hstack((XNL_scaled, algae_clean_df[categorical_var].to_numpy()))

X_algae.shape

In [ ]:
pd.DataFrame(X_algae).head()

In [ ]:
XNL_algae.shape

In [ ]:
pd.DataFrame(XNL_algae).head()

In [ ]:
# build the linear model

graphics_path = '/Users/donaldbrown/Dropbox/department/Classes/Code4Classes/DS6040Code/ClassFigures/FiguresDS6040F23/'

a1_mean = log_a1.mean()
k = X_algae.shape[1]

with pm.Model() as algae_lin_model:

    #Data
    X = pm.MutableData("X", X_algae) # training data
    y = pm.MutableData("y", log_a1)

    #Regression model parameters
    α = pm.Normal('α', mu = a1_mean, sigma = 1) # informed prior
    β = pm.MvNormal('β', mu=np.zeros(k), cov= np.eye(k), shape=k, dims ='Predictors') # informed prirs

    #Likelihood parameters
    nu = pm.Exponential('nu', 1/29.0)+ 1 # uninformed prior
    # σ = pm.Gamma('σ', 2.0, 2.0)
    σ = pm.HalfStudentT('σ',2,8)
    μ = α + pm.math.dot(X,β)
    log_algae = pm.StudentT('log_algae', nu=nu, mu=μ, sigma=σ, observed=y)

g1 = pm.model_to_graphviz(algae_lin_model)
g1.render(graphics_path+"algae_lin_model", format="png", cleanup=True)

In [ ]:
with algae_lin_model:
    trace= pm.sample(random_seed = random_seed, return_inferencedata=False)
    algae_lin_trace = pm.to_inference_data(trace=trace, log_likelihood=True)

In [ ]:
az.plot_trace(algae_lin_trace, kind = 'rank_bars', compact = False)

In [ ]:
az.summary(algae_lin_trace)

In [ ]:
a1_mean = log_a1.mean()
k = XNL_algae.shape[1]

with pm.Model() as algae__model:

    #Data
    X = pm.MutableData("X", X_algae) # training data
    y = pm.MutableData("y", log_a1)

    #Regression model parameters
    α = pm.Normal('α', mu = a1_mean, sigma = 1) # informed prior
    β = pm.MvNormal('β', mu=np.zeros(k), cov= np.eye(k), shape=k, dims ='Predictors') # informed prirs

    #Likelihood parameters
    nu = pm.Exponential('nu', 1/29.0)+ 1 # uninformed prior
    # σ = pm.Gamma('σ', 2.0, 2.0)
    σ = pm.HalfStudentT('σ',2,8)
    μ = α + pm.math.dot(X,β)
    log_algae = pm.StudentT('log_algae', nu=nu, mu=μ, sigma=σ, observed=y)

pm.model_to_graphviz(algae_lin_model)

In [ ]:
# build the nonlinear model

a1_mean = log_a1.mean()
k = XNL_algae.shape[1]

with pm.Model() as algae_nl_model:

    #Data
    X = pm.MutableData("X", XNL_algae) # training data
    y = pm.MutableData("y", log_a1)

    #Regression model parameters
    α = pm.Normal('α', mu = a1_mean, sigma = 1) # informed prior
    β = pm.MvNormal('β', mu=np.zeros(k), cov= np.eye(k), shape=k, dims ='Predictors') # informed prirs

    #Likelihood parameters
    nu = pm.Exponential('nu', 1/29.0)+ 1 # uninformed prior
    # σ = pm.Gamma('σ', 2.0, 2.0)
    σ = pm.HalfStudentT('σ',2,8)
    μ = α + pm.math.dot(X,β)
    log_algae = pm.StudentT('log_algae', nu=nu, mu=μ, sigma=σ, observed=y)

pm.model_to_graphviz(algae_nl_model)

In [ ]:
with algae_nl_model:
    trace= pm.sample(random_seed = random_seed, return_inferencedata=False)
    algae_nl_trace = pm.to_inference_data(trace=trace, log_likelihood=True)

In [ ]:
az.plot_trace(algae_nl_trace, kind = 'rank_bars', compact = False)

In [ ]:
az.summary(algae_nl_trace)

In [ ]:
# random_seed = 135573

# spring_obs = a.loc[:, "spring"]
# summer_obs = a.loc[:, "summer"]
# winter_obs = a.loc[:, "winter"]
# medium_obs = a.loc[:, "medium"]
# small_obs = a.loc[:, "mxPH"]
# l_obs = a.loc[:, "l"]
# m_obs = a.loc[:, "m"]
# mxPH_obs = a.loc[:, "mxPH"]
# mnO2_obs = a.loc[:, "mnO2"]
# Cl_obs = a.loc[:, "Cl"]
# NO3_obs = a.loc[:, "NO3"]
# NH4_obs = a.loc[:, "NH4"]
# oPO4_obs = a.loc[:, "oPO4"]
# PO4_obs = a.loc[:, "PO4"]
# Chla_obs = a.loc[:, "Chla"]
# a1_obs = a.loc[:, "a1"]

# with pm.Model() as a_model:

#     spring = pm.Data("spring", spring_obs)
#     summer = pm.Data("summer", summer_obs)
#     winter = pm.Data("winter", winter_obs)
#     medium = pm.Data("medium", medium_obs)
#     small = pm.Data("small", small_obs)
#     l = pm.Data("l", l_obs)
#     m = pm.Data("m", m_obs)
#     mxPH = pm.Data("mxPH", mxPH_obs)
#     mnO2 = pm.Data("mnO2", mnO2_obs)
#     Cl = pm.Data("Cl", Cl_obs)
#     NO3 = pm.Data("NO3", NO3_obs)
#     NH4 = pm.Data("NH4", NH4_obs)
#     oPO4 = pm.Data("oPO4", oPO4_obs)
#     PO4 = pm.Data("PO4", PO4_obs)
#     Chla = pm.Data("Chla", Chla_obs)

#     nu = pm.Exponential("nu", 1/20)
#     sigma = pm.Uniform("sigma", 0, 10)
#     beta_0 = pm.Normal("beta_0", 0, 10)
#     beta_spring = pm.Normal("beta_spring", 0, 10)
#     beta_summer = pm.Normal("beta_summer", 0, 10)
#     beta_winter = pm.Normal("beta_winter", 0, 10)
#     beta_medium = pm.Normal("beta_medium", 0, 10)
#     beta_small = pm.Normal("beta_small", 0, 10)
#     beta_l = pm.Normal("beta_l", 0, 10)
#     beta_m = pm.Normal("beta_m", 0, 10)
#     beta_mxPH = pm.Normal("beta_mxPH", 0, 10)
#     beta_mnO2 = pm.Normal("beta_mnO2", 0, 10)
#     beta_Cl = pm.Normal("beta_Cl", 0, 10)
#     beta_NO3 = pm.Normal("beta_NO3", 0, 10)
#     beta_NH4 = pm.Normal("beta_NH4", 0, 10)
#     beta_oPO4 = pm.Normal("beta_oPO4", 0, 10)
#     beta_PO4 = pm.Normal("beta_PO4", 0, 10)
#     beta_Chla = pm.Normal("beta_Chla", 0, 10)

#     mu = pm.Deterministic("mu", beta_0 + \
#                          beta_spring * spring + beta_summer * summer + \
#                          beta_winter * winter + \
#                          beta_medium * medium + beta_small * small + \
#                          beta_l * l + beta_m * m + \
#                          beta_mxPH * mxPH + beta_mnO2 * mnO2 + \
#                          beta_Cl * Cl + beta_NO3 * NO3 + beta_NH4 * NH4 + \
#                          beta_oPO4 * oPO4 + beta_PO4 * PO4 + beta_Chla * Chla)

#     a1 = pm.StudentT("a1", mu=mu, sigma=sigma, nu=nu, observed = a1_obs)

#     a_trace = pm.sample(target_accept=0.9, random_seed=random_seed, return_inferencedata=False)

### Answer 2.C

I chose these priors with these parameters because the distribution plots in 2.A seemed to be fairly normal distributions.

(d) Evaluate the sampling and choice of priors using Bayesian p-value
and posterior predictive plots. Briefly explain your results.

In [ ]:
algae_pp = pm.sample_posterior_predictive(algae_lin_trace,
                                          model = algae_lin_model, random_seed = random_seed)

In [ ]:
algae_nl_pp = pm.sample_posterior_predictive(algae_nl_trace,
                                          model = algae_nl_model, random_seed = random_seed)

In [ ]:
az.plot_bpv(algae_pp)

In [ ]:
az.plot_bpv(algae_nl_pp)

In [ ]:
az.plot_ppc(algae_pp)

In [ ]:
az.plot_ppc(algae_nl_pp)

In [ ]:
# a_pp = pm.sample_posterior_predictive(a_trace, model=a_model, random_seed=random_seed)
# az.plot_bpv(a_pp)

In [ ]:
# az.plot_ppc(a_pp)

(e) Plot the posterior distributions for the parameters and discuss your
results. Plot the response variable vs. one of the predictors using
counterfactuals and discuss this plot.

In [ ]:
az.plot_posterior(
    algae_lin_trace,
    var_names= ['α'], ref_val = a1_mean
);

In [ ]:
az.plot_posterior(
    algae_lin_trace,
    var_names= ['β'], ref_val = 0
);

In [ ]:
az.plot_forest(algae_lin_trace, var_names = ['β'] )

In [ ]:
az.plot_posterior(
    algae_nl_trace,
    var_names= ['α'], ref_val = a1_mean
);

In [ ]:
az.plot_posterior(
    algae_nl_trace,
    var_names= ['β'], ref_val = 0
);

In [ ]:
az.plot_forest(algae_nl_trace, var_names = ['β'] )

In [ ]:
cmp_loo = az.compare({'Linear':algae_lin_trace, 'Nonlinear':algae_nl_trace}, ic='loo')
cmp_loo

In [ ]:
cmp_waic = az.compare({'Linear':algae_lin_trace, 'Nonlinear':algae_nl_trace}, ic='loo')
cmp_waic

In [ ]:
# with a_model:
#     az.plot_posterior(trace_robust,
#                       var_names = ["beta_0", "beta_spring",
#                                    "beta_summer", "beta_winter",
#                                    "beta_medium", "beta_small",
#                                    "beta_l", "beta_m", "beta_mxPH", "beta_mnO2",
#                                    "beta_Cl", "beta_NO3", "beta_NH4",
#                                    "beta_oPO4", "beta_Chla"],
#                       ref_val=0)

In [ ]:
# with pm.Model() as a_pred_model:
#   beta0 = pm.Normal('beta0', 0, 10)
#   betaNO3 = pm.Normal('NO3', 0, 10)

#   nu = pm.Exponential('nu', 1/30)
#   sigma = pm.Uniform('sigma', 0, 10)

#   zNO3 = pm.MutableData('zNO3', NO3_obs, dims='obs_id')

#   mu = pm.Deterministic('mu', beta0+betaNO3*zNO3)

#   a_zScores = pm.StudentT('a_zScores', nu=nu, mu=mu, sigma=sigma,
#                           observed=a1_obs)

#   a_pred_trace = pm.sample(random_seed=random_seed)

In [ ]:
# with a_pred_model:
#   a_ppc = pm.sample_posterior_predictive(a_pred_trace,
#                                          var_names=['mu'],
#                                          random_seed=random_seed)

In [ ]:
# new_NO3 = np.linspace(-2, 2, num=200)

In [ ]:
# with a_model:
#   pm.set_data({'new_NO3':new_NO3})

#   a_pp_pred = pm.sample_posterior_predictive(a_pred_trace,
#                                              var_names=["mu"],
#                                              return_inferencedata=True,
#                                              predictions=True,
#                                              extend_interfacedata=True,
#                                              random_seed=random_seed)

In [ ]:
# _, ax = plt.subplots(figsize=(12,6))

# preds_counterfactual = a_pp_pred.predictions_constant_data["new_NO3"]
# model_preds = a_pp_pred.predictions

# ax.vlines(preds_counterfactual,
#           *az.hdi(model_preds)["mu"].transpose("hdi", ...),
#           alpha=0.5)

# ax.plot(preds_counterfactual,
#         model_preds['mu'].mean(("chain","draw")),
#         "sigma",
#         ms=5,
#         color="C1",
#         alpha=0.5,
#         label="Expected prob.")

(f) The data set, algaetest.csv, contains a test set. Build a second model
that may contain fewer predictors but should contain at least one
nonlinear element. Evaluate both models using ROC curves for the
test set. For the better performing model show the parameter posterior distributions.

In [ ]:
algae_new_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/algaeTest.csv", header = None, names=col_names, na_values =['XXXXXXX'])

In [ ]:
# Clean and process the test data
# create the response variable and drop the other algae variables

log_a1_test = np.log10(algae_new_df.a1+1)
algae_new_df.drop(['a1','a2','a3','a4','a5','a6','a7'], axis = 1, inplace = True)

# drop oPO4

algae_new_df.drop(['oPO4'], axis = 1, inplace = True)

# drop NO3

# replace NO3 with 0's

algae_new_df['NO3'] = np.zeros(algae_new_df.shape[0])

In [ ]:
# Code the categorical variables
# Use one-hot encoding

algae_test_df = pd.get_dummies(algae_new_df, columns=['season','size','speed'], drop_first = False, dtype = float)

algae_test_df.describe(include = 'all')

In [ ]:
# Impute missing values

algae_test_df = DataFrameImputer().fit_transform(algae_test_df)
algae_test_df.isnull().sum()

In [ ]:
# Create log transformed predictors for a nonlinear model
nl_set = ['Cl','PO4','Chla']
nl_names = ['Cl_nl','PO4_nl','Chla_nl']

for n,m in zip(nl_set,nl_names):
    algae_test_df[m] = np.log(algae_test_df[n]+1)

In [ ]:
#Scale the test variable
#linear model

numeric_var = ['mxPH', 'mnO2', 'Cl', 'NO3', 'NH4', 'PO4', 'Chla']
scaler = StandardScaler()
X_test = scaler.fit_transform(algae_test_df[numeric_var])
stats.describe(X_test)

In [ ]:
#Nonlinear variable scaling
# Use Z-scaling

nl_numeric_var = ['mxPH', 'mnO2', 'Cl_nl', 'NO3', 'NH4', 'PO4_nl', 'Chla_nl']
scaler = StandardScaler()
XNL_test = scaler.fit_transform(algae_test_df[nl_numeric_var])
stats.describe(XNL_test)

In [ ]:
# Data for linear regression
categorical_var = algae_test_df.iloc[:,7:17].columns
X_algae_test = np.hstack((X_test, algae_test_df[categorical_var].to_numpy()))

In [ ]:
# Data for nonlinear regression

XNL_algae_test = np.hstack((XNL_test, algae_test_df[categorical_var].to_numpy()))

In [ ]:
# Test data predictions for the linear model

with algae_lin_model:
    # Test data
    pm.set_data({"X": X_algae_test, "y":log_a1_test})
    #Predict outcomes & probabilities
    algae_lin_test_pp = pm.sample_posterior_predictive(
        algae_lin_trace,
        var_names = ["log_algae"],
        return_inferencedata=True,
        predictions=True,
        extend_inferencedata=True,
        random_seed = random_seed)

In [ ]:
algae_lin_test_pp

In [ ]:
# MSE for the linear model

pred = algae_lin_test_pp.predictions["log_algae"].mean(dim =['chain','draw'])
mse = mean_squared_error(pred, log_a1_test)
# nmse = norm_mse(y_pred, y_test)
print("Mean Square Error: %0.2f" % (mse))
# print("Normalized Mean Square Error: %0.2f" % (nmse))

In [ ]:
# Test data predictions for the nonlinear model

with algae_nl_model:
    # Test data
    pm.set_data({"X": XNL_algae_test, "y":log_a1_test})
    #Predict outcomes & probabilities
    algae_nl_test_pp = pm.sample_posterior_predictive(
        algae_nl_trace,
        var_names = ["log_algae"],
        return_inferencedata=True,
        predictions=True,
        extend_inferencedata=True,
        random_seed = random_seed)

In [ ]:
# MSE for the nonlinear model

pred = algae_nl_test_pp.predictions["log_algae"].mean(dim =['chain','draw'])
mse = mean_squared_error(pred, log_a1_test)
# nmse = norm_mse(y_pred, y_test)
print("Mean Square Error: %0.2f" % (mse))
# print("Normalized Mean Square Error: %0.2f" % (nmse))

In [ ]:
# Plot of predictions linear model

_, ax = plt.subplots(figsize=(12, 6))

pred_mean = algae_lin_test_pp.predictions["log_algae"].mean(dim=["draw", "chain"])
ax.scatter(X_test[:,5],pred_mean, label = 'Predictions')
ax.scatter(X_test[:,5], log_a1_test, label = 'Test Set')
ax.vlines(
    X_test[:,5],
    *az.hdi(algae_lin_test_pp.predictions)["log_algae"].transpose("hdi", ...),
    alpha=0.8,
)
ax.set_ylabel('Log a1 Algae')
ax.set_xlabel('Z-scaled PO4')
ax.set_title('Linear Model Predictions and Test Set Values')
ax.legend(loc='upper right')

In [ ]:
_, ax = plt.subplots(figsize=(12, 6))

pred_mean = algae_lin_test_pp.predictions["log_algae"].mean(dim=["draw", "chain"])
plt.scatter(X_test[:,6],pred_mean, label='Predictions')
plt.scatter(X_test[:,6], log_a1_test, label='Test Set')
ax.vlines(
    X_test[:,6],
    *az.hdi(algae_lin_test_pp.predictions)["log_algae"].transpose("hdi", ...),
    alpha=0.8,
)

ax.set_ylabel('Log a1 Algae')
ax.set_xlabel('Z-scaled Chla')
ax.set_title('Linear Model Predictions and Test Set Values')
ax.legend(loc='upper right')

In [ ]:
# Plot of predictions NL Model

_, ax = plt.subplots(figsize=(12, 6))

pred_mean = algae_nl_test_pp.predictions["log_algae"].mean(dim=["draw", "chain"])
ax.scatter(X_test[:,5],pred_mean, label = 'Predictions')
ax.scatter(X_test[:,5], log_a1_test, label = 'Test Set')
ax.vlines(
    X_test[:,5],
    *az.hdi(algae_nl_test_pp.predictions)["log_algae"].transpose("hdi", ...),
    alpha=0.8,
)
ax.set_ylabel('Log a1 Algae')
ax.set_xlabel('Z-scaled PO4')
ax.set_title('Nonlinear Model Predictions and Test Set Values')
ax.legend(loc='upper right')

In [ ]:
# Plot of predictions NL Model Chla

_, ax = plt.subplots(figsize=(12, 6))

pred_mean = algae_nl_test_pp.predictions["log_algae"].mean(dim=["draw", "chain"])
ax.scatter(X_test[:,6],pred_mean, label = 'Predictions')
ax.scatter(X_test[:,6], log_a1_test, label = 'Test Set')
ax.vlines(
    X_test[:,6],
    *az.hdi(algae_nl_test_pp.predictions)["log_algae"].transpose("hdi", ...),
    alpha=0.8,
)
ax.set_ylabel('Log a1 Algae')
ax.set_xlabel('Z-scaled Chla')
ax.set_title('Nonlinear Model Predictions and Test Set Values')
ax.legend(loc='upper right')

In [ ]:
# random_seed = 135573

# spring_obs = aT.loc[:, "spring"]
# summer_obs = aT.loc[:, "summer"]
# winter_obs = aT.loc[:, "winter"]
# mxPH_obs = aT.loc[:, "mxPH"]
# mnO2_obs = aT.loc[:, "mnO2"]
# Cl_obs = aT.loc[:, "Cl"]
# NO3_obs = aT.loc[:, "NO3"]
# NH4_obs = aT.loc[:, "NH4"]
# PO4_obs = aT.loc[:, "PO4"]
# Chla_obs = aT.loc[:, "Chla"]
# a1_obs = aT.loc[:, "a1"]

# with pm.Model() as a_model:

#     spring = pm.Data("spring", spring_obs)
#     summer = pm.Data("summer", summer_obs)
#     winter = pm.Data("winter", winter_obs)
#     mxPH = pm.Data("mxPH", mxPH_obs)
#     mnO2 = pm.Data("mnO2", mnO2_obs)
#     Cl = pm.Data("Cl", Cl_obs)
#     NO3 = pm.Data("NO3", NO3_obs)
#     NH4 = pm.Data("NH4", NH4_obs)
#     PO4 = pm.Data("PO4", PO4_obs)
#     Chla = pm.Data("Chla", Chla_obs)

#     nu = pm.Exponential("nu", 1/20)
#     sigma = pm.Uniform("sigma", 0, 10)
#     beta_0 = pm.Normal("beta_0", 0, 10)
#     beta_spring = pm.Normal("beta_spring", 0, 10)
#     beta_summer = pm.Normal("beta_summer", 0, 10)
#     beta_winter = pm.Normal("beta_winter", 0, 10)
#     beta_mxPH = pm.Normal("beta_mxPH", 0, 10)
#     beta_mnO2 = pm.Normal("beta_mnO2", 0, 10)
#     beta_Cl = pm.Normal("beta_Cl", 0, 10)
#     beta_NO3 = pm.Normal("beta_NO3", 0, 10)
#     beta_NH4 = pm.Normal("beta_NH4", 0, 10)
#     beta_PO4 = pm.Normal("beta_PO4", 0, 10)
#     beta_Chla = pm.Normal("beta_Chla", 0, 10)

#     mu = pm.Deterministic("mu", beta_0 + \
#                          beta_spring * spring + beta_summer * summer + \
#                          beta_winter * winter + \
#                          beta_mxPH * mxPH + beta_mnO2 * mnO2 + \
#                          beta_Cl * Cl + beta_NO3 * NO3 + beta_NH4 * NH4 + \
#                          beta_PO4 * PO4 + beta_Chla * Chla)

#     a1 = pm.StudentT("a1", mu=mu, sigma=sigma, nu=nu, observed = a1_obs)

#     a_trace = pm.sample(draws=2000, tune=4000, cores = 8,
#                         random_seed=random_seed, return_inferencedata = False)

#     trace_robust = pm.to_inference_data(trace=a_trace, log_likelihood=True)

#     a_pp_sqrt = pm.sample_posterior_predictive(trace_robust,
#                                            random_seed = random_seed)

(g) Summarize your results and give your answer to the EU’s goal.

## Question 3

(30) The South African Health Ministry is now asking you to to extend
your work from problem 1 and perform a full analysis of the data in
the coronary heart disease study [1]. Their goal is to understand factors
that may be associate with coronary heart disease to inform further more focused research on those factors. For this work you will again use the
CHD data set (CHDdata.csv ). Perform the following steps and answer
the associated questions.

(a) Perform a simple exploratory analysis of the data by at least viewing
summary statistics. Comment on what the EDA implies for your
analysis.

In [ ]:
data1.head()

In [ ]:
data1.describe()

In [ ]:
sns.pairplot(data1,vars= ['sbp', 'tobacco', 'ldl', 'adiposity', 'typea', 'obesity', 'alcohol', 'age'], hue='chd')

In [ ]:
data1.corr()

In [ ]:
az.plot_dist(data1.chd, kind = 'hist')

In [ ]:
# summary_stats = (data1.loc[:, ['sbp', 'tobacco', 'ldl', 'adiposity', 'typea', 'obesity',
#        'alcohol', 'age']]
#                          .agg(["mean", "std", "count"]))
# summary_stats

In [ ]:
# df = data1.loc[:, ['sbp', 'tobacco', 'ldl', 'adiposity', 'typea', 'obesity',
#        'alcohol', 'age']]
# df = df.melt(var_name='cols',  value_name='vals')

# sns.displot(kind='kde', data=df, col='cols', col_wrap=4, x='vals', hue="cols", facet_kws={'sharey': False, 'sharex': False})

In [ ]:
# df = data1[["famhist", "chd"]]
# sns.countplot(x="famhist", hue="chd", data=data1)

(b) Preprocess the data by scaling the predictor variables using a ztransformation (subtract the mean divide by the standard deviation).
Explain why this is appropriate. Code the categorical variables and
explain your coding.

In [ ]:
# Impute missing values

data1_model_df = pd.get_dummies(data1, drop_first = False, dtype = float)

data1_clean_df = DataFrameImputer().fit_transform(data1_model_df)
data1_clean_df.isnull().sum()

In [ ]:
data1_clean_df.describe()

### Answer 3.B

We can scale with this method because

(c) Develop and use Bayesian GLM model with all the predictor variables. Explain your choice of priors and their parameters. Show the
graphical representation of your model.

In [ ]:
data1_clean_df.columns

In [ ]:
# with pm.Model() as glm_model:
#   Y = data1_clean_df["chd"]
#   X = data1_clean_df.loc[:,['sbp', 'tobacco', 'ldl', 'adiposity', 'typea', 'obesity', 'alcohol',
#                              'age', 'famhist_Absent','famhist_Present']] #predictors
#   k = X.shape[1] # no. of predictors
#   alpha = pm.Normal('alpha', 0, 100)

#   beta = pm.MvNormal('beta', mu=np.zeros(k), cov=100*np.eye(k), shape=k)
#   mu = alpha + pm.math.dot(X,beta)
#   p = pm.Deterministic('p', pm.invlogit(mu))
#   Yl = pm.Bernoulli('Yl', p=p, observed=Y)

# pm.model_to_graphviz(glm_model)

In [ ]:
with pm.Model() as glm_model:
  Y = data1_clean_df["chd"]
  alpha = pm.Normal('alpha', mu=0, sigma=5)
  beta1 = pm.Normal('beta1', mu=0, sigma=5)
  beta2 = pm.Normal('beta2', mu=0, sigma=5)
  beta3 = pm.Normal('beta3', mu=0, sigma=5)
  beta4 = pm.Normal('beta4', mu=0, sigma=5)
  beta5 = pm.Normal('beta5', mu=0, sigma=5)
  beta6 = pm.Normal('beta6', mu=0, sigma=5)
  beta7 = pm.Normal('beta7', mu=0, sigma=5)
  beta8 = pm.Normal('beta8', mu=0, sigma=5)
  beta9 = pm.Normal('beta9', mu=0, sigma=5)
  beta10 = pm.Normal('beta10', mu=0, sigma=5)

  mu = alpha + beta1*data1_clean_df["sbp"] + beta2*data1_clean_df['tobacco']+\
  beta3*data1_clean_df["ldl"] + beta4*data1_clean_df["adiposity"] + \
  beta5*data1_clean_df["typea"] + beta6*data1_clean_df["obesity"] + \
  beta7*data1_clean_df["alcohol"] + beta8*data1_clean_df["age"] + beta9*data1_clean_df["famhist_Absent"] + \
  beta10*data1_clean_df["famhist_Present"]

  p = pm.Deterministic('p', pm.invlogit(mu))

  Yl = pm.Bernoulli('Yl', p=p, observed=Y)

pm.model_to_graphviz(glm_model)

In [ ]:
with glm_model:
  glm_chd_trace = pm.sample(1000, tune=2000, random_seed = random_seed, cores = 8, target_accept=0.9)

In [ ]:
az.plot_trace(glm_chd_trace, compact = False, var_names = ['alpha','beta1','beta2',
                                                           'beta3','beta4','beta5',
                                                           'beta6','beta7','beta8',
                                                           'beta9','beta10'], figsize = (18,15))

In [ ]:
az.summary(glm_chd_trace, var_names= ['alpha','beta1','beta2',
                                  'beta3','beta4','beta5',
                                  'beta6','beta7','beta8',
                                  'beta9','beta10'])

(d) Evaluate both the sampling used by your model, as well as, the prior
and posterior predictive results.


In [ ]:
glm_chd_pp = pm.sample_posterior_predictive(glm_chd_trace, model = glm_model, random_seed = random_seed)

In [ ]:
az.plot_bpv(glm_chd_pp)

(e) Provide plots of the posterior distributions for the parameters and
discuss what these results imply for the relevant predictor variables
and the overall model. Show posterior analyses to include relevant
odds ratio(s) and describe what this analysis means for the goal of
your study.

In [ ]:
az.plot_posterior(glm_chd_trace,
                  var_names=['alpha','beta1','beta2',
                             'beta3','beta4','beta5',
                             'beta6','beta7','beta8',
                             'beta9','beta10'],
                  ref_val = 0)

In [ ]:
all_predictors = ['sbp', 'tobacco', 'ldl', 'adiposity', 'typea', 'obesity', 'alcohol',
                  'age', 'famhist_Absent', 'famhist_Present']
parameters = ['alpha','beta1','beta2','beta3','beta4','beta5',
              'beta6','beta7','beta8','beta9','beta10']

fig, axes = plt.subplots(3,2, figsize=(12,8), dpi=150)

for ax, estimate, title, xlabel in zip(fig.axes, (glm_chd_trace.posterior[i] for i in parameters),
 ['Intercept'] + all_predictors, ['Intercept'] + all_predictors):

  pm.plot_posterior(estimate, point_estimate='mode', ax=ax)
  ax.set_title(title, fontsize=12)
  ax.set_xlabel(xlabel, fontsize=12)

fig.delaxes(axes[2,1])
plt.tight_layout()

In [ ]:
for i in parameters:
  az.plot_posterior(glm_chd_trace.posterior[i], ref_val = 0)

In [ ]:
az.plot_forest(glm_chd_trace, var_names=parameters)

In [ ]:
az.plot_ppc(glm_chd_pp)

In [ ]:
_, ax = plt.subplots(figsize=(12, 6))

b = glm_chd_trace.posterior["beta3"]
OR = np.mean(b, axis =0)
lb, ub = np.percentile(OR, 2.5), np.percentile(OR, 97.5)
plt.hist(np.exp(OR), bins=20, density=True)
plt.axvline(np.exp(lb), color = 'r')
plt.axvline(np.exp(ub), color = 'r')
plt.xlabel("Odds Ratio")
plt.title("LDL Odds Ratio with 95% CI")

(f) Develop a second model to help in your study of this problem. This
new model can use fewer predictor variables but must contain at
least one nonlinear component. Explain your choice of priors and
their parameters, and show the graphical representation of your new
model.


In [ ]:
data1_clean_df["age_sq"] = data1_clean_df.age**2

In [ ]:
with pm.Model() as nl_model:
  Y = data1_clean_df["chd"]
  alpha = pm.Normal('alpha', 0, 100)
  beta1 = pm.Normal('beta1', mu=0, sigma=10)
  beta2 = pm.Normal('beta2', mu=0, sigma=10)
  beta3 = pm.Normal('beta3', mu=0, sigma=10)

  mu = alpha + data1_clean_df['adiposity']*beta1 + data1_clean_df['age']*beta2 + data1_clean_df['age_sq']*beta3
  p = pm.Deterministic('p', pm.invlogit(mu))
  Yl = pm.Bernoulli('Yl', p=p, observed=Y)

pm.model_to_graphviz(nl_model)

In [ ]:
with nl_model:
  nl_chd_trace = pm.sample(1000, tune=2000, random_seed = random_seed, cores = 8)

In [ ]:
az.plot_trace(nl_chd_trace, compact = False, var_names = ['alpha','beta1','beta2',
                                                       'beta3'], figsize = (18,15))

In [ ]:
az.summary(nl_chd_trace, var_names= ['alpha','beta1','beta2',
                                  'beta3'])

(g) For the second model evaluate the sampling and choice of priors using
Bayesian p-value and posterior predictive plots. Briefly explain your
results.

In [ ]:
nl_chd_pp = pm.sample_posterior_predictive(nl_chd_trace, model = nl_model, random_seed = random_seed)

In [ ]:
az.plot_bpv(nl_chd_pp)

(h) For the second model provide plots of the posterior distributions for
the parameters and discuss what these results imply for the relevant
predictor variables and the overall model. Show posterior analyses to
include relevant odds ratio(s) and describe what this analysis means
for the goal of your study.


In [ ]:
az.plot_posterior(nl_chd_trace,
                  var_names=['alpha','beta1','beta2',
                             'beta3'],
                  ref_val = 0)

In [ ]:
all_predictors = ['adiposity',
                  'age', 'age_sq']
parameters = ['alpha','beta1','beta2','beta3']

fig, axes = plt.subplots(3,2, figsize=(12,8), dpi=150)

for ax, estimate, title, xlabel in zip(fig.axes, (nl_chd_trace.posterior[i] for i in parameters),
 ['Intercept'] + all_predictors, ['Intercept'] + all_predictors):

  pm.plot_posterior(estimate, point_estimate='mode', ax=ax)
  ax.set_title(title, fontsize=12)
  ax.set_xlabel(xlabel, fontsize=12)

fig.delaxes(axes[2,1])
plt.tight_layout()

In [ ]:
for i in parameters:
  az.plot_posterior(nl_chd_trace.posterior[i], ref_val = 0)

In [ ]:
az.plot_forest(nl_chd_trace, var_names=parameters)

In [ ]:
az.plot_ppc(nl_chd_pp)

In [ ]:
_, ax = plt.subplots(figsize=(12, 6))

b = nl_chd_trace.posterior["3"]
OR = np.mean(b, axis =0)
lb, ub = np.percentile(OR, 2.5), np.percentile(OR, 97.5)
plt.hist(np.exp(OR), bins=20, density=True)
plt.axvline(np.exp(lb), color = 'r')
plt.axvline(np.exp(ub), color = 'r')
plt.xlabel("Odds Ratio")
plt.title("LDL Odds Ratio with 95% CI")

(i) Compare the two models using leave-one-out cross validation

In [ ]:
cmp_loo = az.compare({'Linear':glm_chd_trace, 'Nonlinear':nl_chd_trace}, ic='loo')
cmp_loo

(j) Briefly summarize your analysis and provide an answer to the South
African Health ministry to the question implied by their goal statement.

## Question 4

(30) You have been asked by the government of Bangladesh to determine
whether the use of contraceptives by women in Bangladesh varies by district. The data come from surveys conducted in Bangladesh.

In [ ]:
bang_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/bangladesh.csv")

In [ ]:
bang_df.head()

In [ ]:
woman = bang_df.woman.values
district = bang_df.district.values
cont = bang_df['use.contraception'].values
children = bang_df['living.children'].values
age = bang_df['age.centered'].values
urban = bang_df.urban.values

(a) Develop three models for these data: pooled, unpooled, and hierarchical for all districts to predict usage of contraceptives. Use only district and age.centered as predictor variables. For each model briefly
explain your choice of priors.

In [ ]:
bang_df['use.contraception'].value_counts()

In [ ]:
# Complete pooling model

with pm.Model() as pooled_model:
    beta0 = pm.Normal("beta0", 0, 10)
    beta1 = pm.Normal("beta1", 0, 10)

    theta = beta0 + beta1*age
    p = pm.Deterministic('p', pm.invlogit(theta))
    Yl = pm.Bernoulli('Yl', p=p, observed=cont)

pm.model_to_graphviz(pooled_model)

In [ ]:
# Unpooled model

district_idx =  bang_df.district.astype("category").cat.codes.values
district_code = bang_df.district.astype("category").cat.categories
n_districts = district_code.size

with pm.Model() as unpooled_model:
  beta0 = pm.Normal("beta0", 0, 10, shape=n_districts)
  beta1 = pm.Normal("beta1", 0, 10, shape=n_districts)

  theta = beta0[district_idx] + beta1[district_idx]*age
  p = pm.Deterministic('p', pm.invlogit(theta))
  Yl = pm.Bernoulli('Yl', p=p, observed=cont)

pm.model_to_graphviz(unpooled_model)

In [ ]:
with pm.Model() as hierarchical_model:
  mu1 = pm.Normal("mu1", 0, 10)
  mu2 = pm.Normal("mu2", 0, 10)

  sigma1 = pm.HalfStudentT("sigma1", 1, 10)
  sigma2 = pm.HalfStudentT("sigma2", 1, 10)

  beta0 = pm.Normal("beta0", mu1, sigma1, shape=n_districts)
  beta1 = pm.Normal("beta1", mu1, sigma1, shape=n_districts)

  theta = beta0[district_idx] + beta1[district_idx]*age
  p = pm.Deterministic('p', pm.invlogit(theta))
  Yl = pm.Bernoulli('Yl', p=p, observed=cont)

pm.model_to_graphviz(hierarchical_model)

(b) For each model evaluate the sampling performance and discuss your
findings.

In [ ]:
# # Prior checks
# pooled_ppc = pm.sample_prior_predictive(model = pooled_model)

In [ ]:
# az.plot_dist(pooled_ppc.prior_predictive['y'].mean(dim=['chain','draw']))

In [ ]:
# az.plot_dist(pooled_ppc.observed_data['y'])

In [ ]:
with pooled_model:
    pooled_trace = pm.sample(1000, cores = 8, tune=1000, random_seed=random_seed)

In [ ]:
az.plot_trace(pooled_trace)

In [ ]:
# posterior predictions
pooled_pp = pm.sample_posterior_predictive(pooled_trace, model = pooled_model)

In [ ]:
az.plot_bpv(pooled_pp)

In [ ]:
az.plot_ppc(pooled_pp)

In [ ]:
with unpooled_model:
    unpooled_trace = pm.sample(1000, cores = 8, tune=1000, random_seed=random_seed)

In [ ]:
pm.plot_trace(unpooled_trace, figsize = (10,8))

In [ ]:
unpooled_pp = pm.sample_posterior_predictive(unpooled_trace, model = unpooled_model)

In [ ]:
az.plot_bpv(unpooled_pp)

In [ ]:
az.plot_ppc(unpooled_pp)

In [ ]:
with hierarchical_model:
    h_trace = pm.sample(1000, cores = 8, tune=1000, random_seed=random_seed)

In [ ]:
pm.plot_trace(h_trace, figsize = (10,8))

In [ ]:
h_pp = pm.sample_posterior_predictive(h_trace, model = hierarchical_model)

In [ ]:
az.plot_bpv(h_pp)

In [ ]:
az.plot_ppc(h_pp)

(c) Plot the posterior distributions for the parameters for each model
and discuss what they show regarding the question posed by WHO.

(d) Plot each of the predictions with age.centered on the x-axis and the
expected proportion of women using contraception on the y-Axis with
overlaid plots for each of the districts, as appropriate. Briefly explain these results as you will report them to the government of
Bangladesh.